## Job links extraction

In [9]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import get_as_dataframe, set_with_dataframe
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.service import Service
import time

# Define scope and credentials
scope = ['https://www.googleapis.com/auth/spreadsheets', "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]
Cred = ServiceAccountCredentials.from_json_keyfile_name('credentials.json', scope)
client = gspread.authorize(Cred)


from urllib.parse import urlparse, parse_qs

def extract_location(url):
    # Parse the URL
    parsed_url = urlparse(url)
    
    # Extract the query parameters
    query_params = parse_qs(parsed_url.query)
    
    # Get the location parameter (after 'l=')
    location = query_params.get('l', [None])[0]
    print("la localisation est :",location)
    return location



# Open or create the spreadsheet
spreadsheet_name = 'try'
spreadsheet = None
try:
    spreadsheet = client.open(spreadsheet_name)
except gspread.exceptions.SpreadsheetNotFound:
    spreadsheet = client.create(spreadsheet_name)

# Get or create sheets
try:
    sheet2 = spreadsheet.worksheet("Feuille 2")
except gspread.exceptions.WorksheetNotFound:
    sheet2 = spreadsheet.add_worksheet(title="Feuille 2", rows="1000", cols="20")

try:
    sheet3 = spreadsheet.worksheet("Feuille 3")
except gspread.exceptions.WorksheetNotFound:
    sheet3 = spreadsheet.add_worksheet(title="Feuille 3", rows="1000", cols="20")

# Set the path to the chromedriver executable
PATH = r"C:\Program Files (x86)\chromedriver.exe"
s = Service(PATH)

# Initialize the Chrome WebDriver
driver = webdriver.Chrome(service=s)

def extract_part(text):
    parts = text.split('=')
    result = f"{parts[0]}={parts[1]}"
    return result

# Initialize a list to store the new job links
job_links = []

# Load existing links
links_processed = []
try:
    existing_df = get_as_dataframe(sheet2)
    if not existing_df.empty:
        links_processed = existing_df["Links"].dropna().tolist()
    print(f"Loaded {len(links_processed)} existing links.")
except Exception as e:
    print(f"An error occurred: {e}")

# Open the webpage
urle ="https://fr.indeed.com/jobs?q=offre+emploi+remote&l=France&from=searchOnDesktopSerp&vjk=564057c05c230ab8" 
driver.get(urle)
country = extract_location(urle)
time.sleep(1)

verif = True
i = 1
while i <= 1 and verif:
    job_container = driver.find_element(By.ID, "mosaic-jobResults")
    jobs = job_container.find_elements(By.TAG_NAME, "li")
    for job in jobs:
        try:
            job_link_element = job.find_element(By.CLASS_NAME, "jcs-JobTitle")
            job_link = job_link_element.get_attribute("href")
            clean_link = extract_part(job_link)
            if clean_link not in links_processed:
                job_links.append(job_link)
                links_processed.append(clean_link)
            else:
                print(f"Link already processed: {clean_link}")
        except NoSuchElementException:
            pass
        except Exception as e:
            print('An error occurred:', str(e))
    
    try:
        button = driver.find_element(By.CSS_SELECTOR, f'a[data-testid="pagination-page-{i + 1}"]')
        driver.execute_script("arguments[0].scrollIntoView(true);", button)
        time.sleep(1)
        button.click()
        time.sleep(2)
    except NoSuchElementException:
        verif = False
    except Exception as e:
        print('An error occurred while clicking the pagination button:', str(e))
        break
    
    i += 1

# Create DataFrames
df_new_links = pd.DataFrame({'Links': job_links})
df_all_links = pd.DataFrame({'Links': links_processed})

# Save new links to Feuille 3
try:
    set_with_dataframe(sheet3, df_new_links)
    print(f"New links saved to 'Feuille 3'.")
except Exception as e:
    print(f"An error occurred while saving new links to 'Feuille 3': {e}")

# Save all links (existing and new) to Feuille 2
try:
    existing_links_df = get_as_dataframe(sheet2)
    combined_df = pd.concat([existing_links_df, df_all_links], ignore_index=True).drop_duplicates().reset_index(drop=True)
    sheet2.clear()
    set_with_dataframe(sheet2, combined_df)
    print(f"All links saved to 'Feuille 2'.")
except Exception as e:
    print(f"An error occurred while saving all links to 'Feuille 2': {e}")

# Close the driver
driver.quit()


Loaded 0 existing links.
la localisation est : France
New links saved to 'Feuille 3'.
All links saved to 'Feuille 2'.


# Job information extraction

In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import get_as_dataframe, set_with_dataframe

# Define scope and credentials
scope = ['https://www.googleapis.com/auth/spreadsheets', "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]
Cred = ServiceAccountCredentials.from_json_keyfile_name('credentials.json', scope)
client = gspread.authorize(Cred)

# Open the existing Google Sheet and get "Feuille 3"
spreadsheet = client.open('try')  # Replace with your Google Sheet name
sheet = spreadsheet.worksheet('Feuille 3')

# Read existing links from the sheet into a DataFrame
existing_df = get_as_dataframe(sheet)

# Extract the 'Links' column
links = existing_df['Links']

# Set the path to the chromedriver executable
PATH = r"C:\Program Files (x86)\chromedriver.exe"
s = Service(PATH)
driver = webdriver.Chrome(service=s)

# Initialize lists to store scraped data
company_name = []
company_link = []
job_location = []
salaire = []
type = []
horaire = []
description = []
job_title = []

try:
    for i, link in enumerate(links):
        print("on est dans ", i)
        # if i > 10 : 
        #     break
        try:
            driver.get(link)
            time.sleep(2)
            job_title.append(driver.find_element(By.CLASS_NAME, "jobsearch-JobInfoHeader-title").text)
            company_link_container = driver.find_element(By.CSS_SELECTOR, 'div[data-testid="jobsearch-CompanyInfoContainer"]')
            company_link_element = company_link_container.find_element(By.TAG_NAME, "a")

            try:
                company_name.append(company_link_container.find_element(By.TAG_NAME, "a").text)
            except NoSuchElementException:
                company_name.append("Null")
            try:
                company_link.append(company_link_element.get_attribute("href"))
            except NoSuchElementException:
                company_link.append("Null")
            try:
                job_location.append(driver.find_element(By.ID, "jobLocationText").text)
            except NoSuchElementException:
                job_location.append("Null")
            try:
                salaire_element = driver.find_element(By.XPATH, '//*[@aria-label="Salaire"]')
                salaire.append(salaire_element.find_element(By.TAG_NAME, "ul").text)
            except NoSuchElementException:
                salaire.append("Null")
            try:
                type_element = driver.find_element(By.XPATH, '//*[@aria-label="Type de poste"]')
                type.append(type_element.find_element(By.TAG_NAME, "ul").text)
            except NoSuchElementException:
                type.append("Null")
            try:
                horaire_element = driver.find_element(By.XPATH, '//*[@aria-label="Temps de travail"]')
                horaire.append(horaire_element.find_element(By.TAG_NAME, "ul").text)
            except NoSuchElementException:
                horaire.append("Null")
            try:
                description_element = driver.find_element(By.ID, 'jobDescriptionText')
                description.append(description_element.text)
            except NoSuchElementException:
                description.append("Null")
        except Exception as e:
            print("error dans ce lien", e)
except Exception as e:
    print("Error in processing link", i, ":", e)

# Ensure all lists have the same length
max_length = max(len(job_title), len(description), len(job_location), len(salaire), len(type), len(horaire), len(company_link), len(company_name))

# Fill shorter lists with "Null"
def fill_list(lst, max_length):
    return lst + ["Null"] * (max_length - len(lst))

job_title = fill_list(job_title, max_length)
description = fill_list(description, max_length)
job_location = fill_list(job_location, max_length)
salaire = fill_list(salaire, max_length)
type = fill_list(type, max_length)
horaire = fill_list(horaire, max_length)
company_link = fill_list(company_link, max_length)
company_name = fill_list(company_name, max_length)

data = pd.DataFrame({
    "Offre d'emploi": job_title,
    "Description de l'emploi": description,
    'Localisation': job_location,
    'Salaire': salaire,
    'Type emploi': type,
    'Horaire': horaire,
    'lien entreprise': company_link,
    'Nom entreprise': company_name
})

# Write the DataFrame to the existing Google Sheet (Feuille 3)
try:
    sheet.clear()
    set_with_dataframe(sheet, data)
    print(f"Data saved to 'Feuille 3'.")
except Exception as e:
    print(f"An error occurred while saving data to 'Feuille 3': {e}")

driver.quit()


on est dans  0
on est dans  1
on est dans  2
on est dans  3
on est dans  4
on est dans  5
on est dans  6
on est dans  7
on est dans  8
error dans ce lien Message: no such element: Unable to locate element: {"method":"css selector","selector":".jobsearch-JobInfoHeader-title"}
  (Session info: chrome=127.0.6533.120); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF61D07EEA2+31554]
	(No symbol) [0x00007FF61CFF7ED9]
	(No symbol) [0x00007FF61CEB872A]
	(No symbol) [0x00007FF61CF08434]
	(No symbol) [0x00007FF61CF0853C]
	(No symbol) [0x00007FF61CF4F6A7]
	(No symbol) [0x00007FF61CF2D06F]
	(No symbol) [0x00007FF61CF4C977]
	(No symbol) [0x00007FF61CF2CDD3]
	(No symbol) [0x00007FF61CEFA33B]
	(No symbol) [0x00007FF61CEFAED1]
	GetHandleVerifier [0x00007FF61D388B1D+3217341]
	GetHandleVerifier [0x00007FF61D3D5AE3+3532675]
	GetHandleVerifier [0x00007FF61D3CB0E0+34891

In [4]:
# Open the existing Google Sheet
spreadsheet = client.open('Indeed')  # Replace with your Google Sheet name
sheet = spreadsheet.worksheet('Feuille 3')
# Read data from the Google Sheet into a DataFrame
df = get_as_dataframe(sheet)
# Replace 'null' with pd.NA
df.replace('null', pd.NA, inplace=True)
# Drop rows where all elements are NaN
df_cleaned = df.dropna(how='all')
# Sort the DataFrame by the 'Nom entreprise' column
df_sorted = df_cleaned.sort_values(by="Nom entreprise")
# Clear the existing sheet content
sheet.clear()
# Write the cleaned and sorted DataFrame back to the Google Sheet
set_with_dataframe(sheet, df_sorted)

# company information

In [17]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException, WebDriverException

import pandas as pd
import time
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import get_as_dataframe, set_with_dataframe

# Define scope and credentials
scope = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('credentials.json', scope)
client = gspread.authorize(creds)

# Open the Google Sheet by name
sheet_name = 'try'
spreadsheet = client.open(sheet_name)

# Access "Feuille 3" for input
try:
    sheet3 = spreadsheet.worksheet('Feuille 3')
except gspread.exceptions.WorksheetNotFound:
    raise Exception("Feuille 3 not found in the spreadsheet")

# Access "Feuille 1" for output
try:
    sheet1 = spreadsheet.worksheet('Feuille 1')
except gspread.exceptions.WorksheetNotFound:
    sheet1 = spreadsheet.add_worksheet(title='Feuille 1', rows=1000, cols=20)

# Read data from "Feuille 3" into a DataFrame
data = get_as_dataframe(sheet3)

# Read existing data from "Feuille 1"
existing_data = get_as_dataframe(sheet1)
existing_data = existing_data.dropna(how='all')  # Drop any completely empty rows

# Add 'Pays' column with pays
data['Pays'] = country

# Extract the desired columns
links = data['lien entreprise']
company_names = data['Nom entreprise']

# List to keep track of processed company names
check_name = []

# Variables to hold the last extracted data
last_extracted_data = {
    "Nom entreprise": "",
    "Description de l'entreprise": "",
    'Site web': "",
    'Secteur': "",
    "Taille de l'entreprise": "",
    'Siège social': "",
    'Profit': "",
    "Année de fondation": ""
}

# Initialize ChromeDriver
PATH = r"C:\Program Files (x86)\chromedriver.exe"
s = Service(PATH)
driver = webdriver.Chrome(service=s)

for i, link in enumerate(links):
    # if i > 10:
    #     break
    company = company_names[i]

    if company in check_name:
        print(f"Company {company} already processed. Filling with last extracted data...")
        # Fill the row with the last extracted data
        data.loc[i, "Description de l'entreprise"] = last_extracted_data["Description de l'entreprise"]
        data.loc[i, 'Site web'] = last_extracted_data['Site web']
        data.loc[i, 'Secteur'] = last_extracted_data['Secteur']
        data.loc[i, "Taille de l'entreprise"] = last_extracted_data["Taille de l'entreprise"]
        data.loc[i, 'Siège social'] = last_extracted_data['Siège social']
        data.loc[i, 'Profit'] = last_extracted_data['Profit']
        data.loc[i, "Année de fondation"] = last_extracted_data["Année de fondation"]
        continue

    try:
        driver.get(link)
    except WebDriverException:
        print(f"Invalid link: {link}. Skipping to the next one.")
        continue
    print("Processing link ", i)
    time.sleep(1)

    company_info = {}

    try:
        company_info['name'] = driver.find_element(By.XPATH, '//*[@itemprop="name"]').text
    except NoSuchElementException:
        print(f"Could not find company name for link: {link}")
        continue

    try:
        founded_element = driver.find_element(By.XPATH, '//*[@data-testid="companyInfo-founded"]')
        company_info['founded'] = founded_element.find_element(By.CLASS_NAME, "css-vjn8gb").text
        print(company_info['founded'])
    except NoSuchElementException:
        print("année not founded")
        company_info['founded'] = "Null"

    try:
        size_element = driver.find_element(By.XPATH, '//*[@data-testid="companyInfo-employee"]')
        print("Element found:", size_element.text)  # Debugging
        company_info['size'] = size_element.find_element(By.CLASS_NAME, "css-1opgcmt").text
        print("Size extracted:", company_info['size'])  # Debugging
    except NoSuchElementException:
        print("Size element not found")  # Debugging
        company_info['size'] = "Null"

    try:
        profit_element = driver.find_element(By.XPATH, '//*[@data-testid="companyInfo-revenue"]')
        company_info['profit'] = profit_element.find_element(By.CLASS_NAME, "css-1opgcmt").text
        print("profit est",company_info['profit'])
    except NoSuchElementException:
        company_info['profit'] = "Null"

    try:
        secteur_element = driver.find_element(By.XPATH, '//*[@data-testid="companyInfo-industry"]')
        company_info['secteur'] = secteur_element.find_element(By.CLASS_NAME, "css-vjn8gb").text
        print("le secteur est",company_info['secteur'])
    except NoSuchElementException:
        company_info['secteur'] = "Null"

    try:
        localisation_element = driver.find_element(By.XPATH, '//*[@data-testid="companyInfo-headquartersLocation"]')
        company_info['localisation'] = localisation_element.find_element(By.TAG_NAME, "span").text
    except NoSuchElementException:
        company_info['localisation'] = "Null"

    try:
        website_element = driver.find_element(By.XPATH, '//*[@data-testid="companyInfo-companyWebsite"]')
        company_info['website'] = website_element.find_element(By.TAG_NAME, "a").get_attribute("href")
    except NoSuchElementException:
        company_info['website'] = "Null"

    try:
        description_element = driver.find_element(By.XPATH, '//*[@data-testid="less-text"]')
        description_less = description_element.text
        try:
            driver.execute_script("arguments[0].scrollIntoView(true);", description_element)
            time.sleep(1)
            load_more = description_element.find_element(By.TAG_NAME, 'button')
            load_more.click()
        except:
            print('no load more')
        try:
            description_more = driver.find_element(By.XPATH, '//*[@data-testid="more-text"]').text
            company_info['description'] = description_more
        except NoSuchElementException:
            print("no description more")
            company_info['description'] = description_less
    except NoSuchElementException:
        company_info['description'] = "Null"

    # Store the scraped data in the last_extracted_data dictionary
    last_extracted_data = {
        "Nom entreprise": company_info['name'],
        "Description de l'entreprise": company_info['description'],
        'Site web': company_info['website'],
        'Secteur': company_info['secteur'],
        "Taille de l'entreprise": company_info['size'],
        'Siège social': company_info['localisation'],
        'Profit': company_info['profit'],
        "Année de fondation": company_info['founded']
    }

    # Fill the row with the extracted data
    data.loc[i, "Description de l'entreprise"] = last_extracted_data["Description de l'entreprise"]
    data.loc[i, 'Site web'] = last_extracted_data['Site web']
    data.loc[i, 'Secteur'] = last_extracted_data['Secteur']
    data.loc[i, "Taille de l'entreprise"] = last_extracted_data["Taille de l'entreprise"]
    data.loc[i, 'Siège social'] = last_extracted_data['Siège social']
    data.loc[i, 'Profit'] = last_extracted_data['Profit']
    data.loc[i, "Année de fondation"] = last_extracted_data["Année de fondation"]

    # Add the company name to check_name
    check_name.append(company)


# Créer une fonction pour mapper les secteurs aux catégories
def map_to_category(sector):
    if sector in [
    'Informatique',
    "Services d'assistance informatique",
    'Services web',
    'Matériel informatique',
    'Sécurité informatique et des réseaux',
    'Développement de logiciels',
    'Technologies et services de l’information',
    'Services et conseil en informatique',
    'Télécommunications',
    'Technologie, information et Internet',
    'Produits de réseaux informatiques',
    'Fabrication de produits informatiques et électroniques',
    'Services de marketing',
    'Jeux vidéo',
    'Services de blockchain'
]:
        return 'IT'
    elif sector in [
        'Services bancaires', 'Activités de placement pour compte propre',
        'Services financiers', 'Assurances', 'Comptabilité','Finance','Services bancaires et de prêts','Traitement des transactions financières',
    ]:
        return 'Finance'
    elif sector in [
    'Services de bien-être et installations sportives',
    'Hôpitaux et services de santé',
    'Soins de santé mentale',
    'Fabrication d’équipements médicaux',
    'Recherche en biotechnologie',
    'Cosmétiques'
]:
        return 'Santé'
    elif sector in [
    'Pharmaceutique et biotechnologie',
    'Programmes d’administration de l’éducation',
    'Enseignement primaire et secondaire',
    'Enseignement supérieur',
    'Enseignement',
    'Fournisseurs d’apprentissage en ligne'
    'Enseignement et formation',"Services d'enseignement et de formation"
]:
        return 'Education'
    else:
        return 'Autre'

# Appliquer la fonction à la colonne "Secteur" pour créer la nouvelle colonne "Category"
data['Categorie'] = data['Secteur'].apply(map_to_category)
# Merge new data with existing data
final_data = pd.concat([existing_data, data], ignore_index=True)


# 
# Appliquer la fonction à chaque ligne du DataFrame
# combined_data['Categorie'] = combined_data['Secteur'].apply(map_to_category)

desired_order = ["Offre d'emploi", "Description de l'emploi",'Localisation','Salaire','Type emploi','Horaire','lien entreprise','Nom entreprise',"Description de l'entreprise",'Site web','Secteur',"Taille de l'entreprise",'Siège social','Profit','Année de fondation','Categorie','Pays']

# Reordering columns
combined_data = final_data[desired_order]


# Write the merged data back to "Feuille 1"
set_with_dataframe(sheet1, combined_data)

# Quit the browser
driver.quit()


Processing link  0
1768
Element found: Taille de l'entreprise
plus de
10 000
Size extracted: plus de
10 000
profit est 4 à 8 Mrd (EUR)
le secteur est Production chimique
Processing link  1
année not founded
Size element not found
Processing link  2
année not founded
Size element not found
le secteur est Médias et communication
Processing link  3
année not founded
Size element not found
le secteur est Ressources humaines et recrutement
no load more
Processing link  4
année not founded
Element found: Taille de l'entreprise
51 à 200
Size extracted: 51 à 200
le secteur est Énergie et exploitation des ressources naturelles
no load more
Processing link  5
1999
Element found: Taille de l'entreprise
plus de
10 000
Size extracted: plus de
10 000
profit est 850 M à 4 Mrd (EUR)
le secteur est Pharmaceutique et biotechnologie
Company Pennylane already processed. Filling with last extracted data...
Processing link  7
année not founded
Element found: Taille de l'entreprise
201 à 500
Size extracted: 

# Supp feuille 3

In [6]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.discovery import build

# Define scope and credentials
scope = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive.file', 'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('credentials.json', scope)
client = gspread.authorize(creds)

# Open the Google Sheet by name
sheet_name = 'try'
spreadsheet = client.open(sheet_name)

# Access "Feuille 3" for input
try:
    sheet3 = spreadsheet.worksheet('Feuille 3')
except gspread.exceptions.WorksheetNotFound:
    raise Exception("Feuille 3 not found in the spreadsheet")

# Clear the contents of "Feuille 3"
sheet3.clear()

print("Le contenu de la feuille 3 a été supprimé avec succès.")


Le contenu de la feuille 3 a été supprimé avec succès.


In [22]:
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import get_as_dataframe, set_with_dataframe
import re

# Define scope and credentials
scope = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('credentials.json', scope)
client = gspread.authorize(creds)

# Open the Google Sheet by name
sheet_name = 'try'
spreadsheet = client.open(sheet_name)

# Access "Feuille 1" for output
try:
    sheet1 = spreadsheet.worksheet('Feuille 1')
except gspread.exceptions.WorksheetNotFound:
    sheet1 = spreadsheet.add_worksheet(title='Feuille 1', rows=1000, cols=20)

# Read data from "Feuille 3" into a DataFrame
data = get_as_dataframe(sheet1)

def process_size(size_str):
    if pd.isna(size_str) or size_str.strip() == "" or size_str.strip().lower() == "null":
        return 0

    # Remove non-breaking spaces (used for thousands separator) and other spaces
    size_str = re.sub(r'\s+', '', size_str.replace("\u00a0", ""))
    
    # Check for intervals like "51 à 200"
    match = re.search(r'(\d+)à(\d+)', size_str)
    if match:
        return int(match.group(2))  # Return the upper bound of the interval
    
    # Check for cases like "plus de 10 000"
    match = re.search(r'plusde(\d+)', size_str.lower())
    if match:
        return int(match.group(1))
    
    # Check for single numbers
    match = re.search(r'(\d+)', size_str)
    if match:
        return int(match.group(1))
    
    return 0

# Add the new column "taille num" based on the "Taille entreprise" column
data['taille num'] = data["Taille de l'entreprise"].apply(process_size)

# Write the updated DataFrame to "Feuille 1"
set_with_dataframe(sheet1, data)
